## Loading data from Amazon S3 bucket
There are two ways to establish access to S3: [IAM roles](https://docs.databricks.com/user-guide/cloud-configurations/aws/iam-roles.html) and access keys.

*Using IAM roles to specify which cluster can access which buckets. Keys can show up in logs and table metadata and are therefore fundamentally insecure.* If you do use keys, you'll have to escape the `/` in your keys with `%2F`.

In [0]:
# File location and type
file_location = "/FileStore/tables/aws_access.csv"
file_type = "csv"

first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df= spark.read.format(file_type) \
.option("header", first_row_is_header) \
.option("sep", delimiter) \
.load(file_location)



In [0]:
from pyspark.sql.functions import *
import urllib
#Access key for amazon S3 bucket
ACCESS_KEY =df.where(col('user_name')=='databricks').select('access_key').collect()[0].access_key
SECRET_KEY =df.where(col('user_name')=='databricks').select('Secret_access_key').collect()[0].Secret_access_key
SECRET_KEY.replace("/","%2F")
ENCODED_SECRET_KEY=urllib.parse.quote(SECRET_KEY,"")

In [0]:
AWS_S3_BUCKET="marketpro786"
MOUNT_NAME="/mnt/marketpro786"
SourceUri="s3a://{0}:{1}@{2}".format(ACCESS_KEY,ENCODED_SECRET_KEY,AWS_S3_BUCKET)

dbutils.fs.mount(SourceUri, MOUNT_NAME) # MOUNTING S3 bucket through Databricks File System (DBFS)
'''
The mount is a pointer to an S3 location, so the data is never synced locally. Once a mount point is created through a cluster, users of that cluster can immediately access the mount point.
'''
#dbutils.fs.unmount("/mnt/marketpro786") For nmounting the S3 bucket

Out[18]: True

In [0]:
#Data can be accessed from S3 bucket through DBFS
%fs ls "/mnt/marketpro786/"

path,name,size
dbfs:/mnt/marketpro786/aisles.csv,aisles.csv,2603
dbfs:/mnt/marketpro786/departments.csv,departments.csv,270
dbfs:/mnt/marketpro786/order_products__prior.csv,order_products__prior.csv,577550706
dbfs:/mnt/marketpro786/order_products__train.csv,order_products__train.csv,24680147
dbfs:/mnt/marketpro786/orders.csv,orders.csv,108968645
dbfs:/mnt/marketpro786/products.csv,products.csv,2166953
dbfs:/mnt/marketpro786/sample_submission.csv,sample_submission.csv,1475693


### Reading the data and creating the RDBMS tables

In [0]:
# Import Data
aisles = spark.read.csv("/mnt/marketpro786/aisles.csv",header=True,inferSchema=True)
departments = spark.read.csv("/mnt/marketpro786/departments.csv", header=True, inferSchema=True)
order_products_prior = spark.read.csv("/mnt/marketpro786/order_products__prior.csv", header=True, inferSchema=True)
order_products_train = spark.read.csv("/mnt/marketpro786/order_products__train.csv", header=True, inferSchema=True)
orders = spark.read.csv("/mnt/marketpro786/orders.csv", header=True, inferSchema=True)
products = spark.read.csv("/mnt/marketpro786/products.csv", header=True, inferSchema=True)


In [0]:
# Create Temporary Tables
aisles.createOrReplaceTempView("aisles")
departments.createOrReplaceTempView("departments")
order_products_prior.createOrReplaceTempView("order_products_prior")
order_products_train.createOrReplaceTempView("order_products_train")
orders.createOrReplaceTempView("orders")
products.createOrReplaceTempView("products")


## Exploratory Data Analysis
Exploring imported data using Spark SQL

In [0]:
%sql
show tables

database,tableName,isTemporary
,aisles,true
,baskets,true
,departments,true
,ifthen,true
,mostpopulariteminabasket,true
,order_products_prior,true
,order_products_train,true
,orders,true
,products,true


In [0]:
%sql
select 
  count(order_id) as total_orders, 
  (case 
     when order_dow = '0' then 'Sunday'
     when order_dow = '1' then 'Monday'
     when order_dow = '2' then 'Tuesday'
     when order_dow = '3' then 'Wednesday'
     when order_dow = '4' then 'Thursday'
     when order_dow = '5' then 'Friday'
     when order_dow = '6' then 'Saturday'              
   end) as day_of_week 
  from orders  
 group by order_dow 
 order by total_orders desc
 
/*Showing number of orders placed each day of the week */

total_orders,day_of_week
600905,Sunday
587478,Monday
467260,Tuesday
453368,Friday
448761,Saturday
436972,Wednesday
426339,Thursday


In [0]:
%sql
select 
  count(order_id) as total_orders, 
  order_hour_of_day as hour 
  from orders 
 group by order_hour_of_day 
 order by order_hour_of_day
 
 /*Showing number of orders placed each hour of the day */

total_orders,hour
22758,0
12398,1
7539,2
5474,3
5527,4
9569,5
30529,6
91868,7
178201,8
257812,9


In [0]:
%sql
select countbydept.*
  from (
  -- from product table, let's count number of records per dept
  -- and then sort it by count (highest to lowest) 
  select department_id, count(1) as counter
    from products
   group by department_id
   order by counter asc 
  ) as maxcount
inner join (
  -- let's repeat the exercise, but this time let's join
  -- products and departments tables to get a full list of dept and 
  -- prod count
  select
    d.department_id,
    d.department,
    count(1) as products
    from departments d,products p
      where p.department_id = d.department_id
   group by d.department_id, d.department 
   order by  products desc
  ) countbydept 
  -- combine the two queries's results by matching the product count
  on countbydept.products = maxcount.counter

department_id,department,products
16,dairy eggs,3449
14,breakfast,1115
3,bakery,1516
21,missing,1258
6,international,1139
1,frozen,4007
11,personal care,6563
9,dry goods pasta,1858
13,pantry,5371
19,snacks,6264


In [0]:
%sql
select count(opp.order_id) as orders, p.product_name as popular_product
  from order_products_prior opp, products p
 where p.product_id = opp.product_id 
 group by popular_product 
 order by orders desc
 limit 10
 --counting orders of each product

orders,popular_product
472565,Banana
379450,Bag of Organic Bananas
264683,Organic Strawberries
241921,Organic Baby Spinach
213584,Organic Hass Avocado
176815,Organic Avocado
152657,Large Lemon
142951,Strawberries
140627,Limes
137905,Organic Whole Milk


In [0]:
%sql
select d.department,count(distinct p.product_id) as products
  from products p,departments d
  where p.department_id=d.department_id
  group by d.department
  order by products desc
  limit 10
  --Counting no. of products in each department

department,products
personal care,6563
snacks,6264
pantry,5371
beverages,4365
frozen,4007
dairy eggs,3449
household,3084
canned goods,2092
dry goods pasta,1858
produce,1684


## Organise and view shopping basket

In [0]:
# Organise the data by shopping basket
from pyspark.sql.functions import collect_set, col, count
rawData = spark.sql("select p.product_name, t.order_id from products p inner join order_products_train t where t.product_id = p.product_id")
baskets = rawData.groupBy('order_id').agg(collect_set('product_name').alias('items'))
baskets.createOrReplaceTempView('baskets')
# Using order_products_train as training data

In [0]:
%sql
select order_id,items from baskets limit 10

order_id,items
1139,"List(Cinnamon Rolls with Icing, Red Vine Tomato, Picnic Potato Salad, Flaky Biscuits, Organic Strawberries, Organic Bakery Hamburger Buns Wheat - 8 CT, Buttermilk Biscuits, Banana, Guacamole)"
1143,"List(Water, Natural Premium Coconut Water, Organic Red Radish, Bunch, Organic Capellini Whole Wheat Pasta, Organic Raspberries, Calming Lavender Body Wash, Organic Garlic, Rustic Baguette, Organic Brussel Sprouts, Organic Butterhead (Boston, Butter, Bibb) Lettuce, Organic Blueberries, Spring Water, Large Lemon, Basil Pesto, Baby Arugula, Organic Hass Avocado, Unscented Long Lasting Stick Deodorant)"
1342,"List(Raw Shrimp, Seedless Cucumbers, Versatile Stain Remover, Organic Strawberries, Organic Mandarins, Chicken Apple Sausage, Pink Lady Apples, Bag of Organic Bananas)"
1468,"List(Pomegranate Seeds, Organic Red Radish, Bunch, Natural Mini Pork Pepperoni, Cage Free Grade AA Large White Eggs, Bartlett Pears, Organic Red Potato, Organic Ginger Root, Banana, Red Peppers, Active Dry Yeast, Organic Lacinato (Dinosaur) Kale, Organic Baby Broccoli, Carrots, Fresh Cauliflower, Organic English Cucumber, Organic Grape Tomatoes, Organic Hass Avocado)"
1591,"List(Cracked Wheat, Strawberry Rhubarb Yoghurt, Organic Bunny Fruit Snacks Berry Patch, Goodness Grapeness Organic Juice Drink, Honey Graham Snacks, Spinach, Granny Smith Apples, Oven Roasted Turkey Breast, Pure Vanilla Extract, Chewy 25% Low Sugar Chocolate Chip Granola, Banana, Original Turkey Burgers Smoke Flavor Added, Twisted Tropical Tango Organic Juice Drink, Navel Oranges, Lower Sugar Instant Oatmeal Variety, Ultra Thin Sliced Provolone Cheese, Natural Vanilla Ice Cream, Cinnamon Multigrain Cereal, Garlic, Goldfish Pretzel Baked Snack Crackers, Original Whole Grain Chips, Medium Scarlet Raspberries, Lemon Yogurt, Original Patties (100965) 12 Oz Breakfast, Nutty Bars, Strawberry Banana Smoothie, Green Machine Juice Smoothie, Coconut Dreams Cookies, Buttermilk Waffles, Uncured Genoa Salami, Organic Greek Whole Milk Blended Vanilla Bean Yogurt)"
1721,"List(Organic Thompson Seedless Raisins, Whole Milk Plain Yogurt, Tomato Paste, Organic Rolled Oats, Organic Reduced Fat Milk, Organic Whole Milk)"
2711,"List(Granny Smith Apples, Alpine Spring Water, Mountain Spring Water, Honeycrisp Apples)"
2888,"List(Shredded Sharp Cheddar Cheese, Organic Raspberries, Clover Honey, Cherubs Heavenly Salad Tomatoes, Whipped Cream Cheese Spread, Ground Cumin, 100% Whole Wheat Bread, Pure & Natural Sour Cream, Organic Milk Whole, Pizza Sauce, 50% Less Sodium Black Beans, Creamy Almond Butter, Organic Grade A Large Brown Eggs, Boneless And Skinless Chicken Breast, Celery Hearts, Low Moisture Part Skim Mozzarella String Cheese, Whole Chicken, Organic Strawberries, Almonds Roasted No Salt, Organic Large Extra Fancy Fuji Apple, Organic Dried Sweetened Cranberries, Whole Kernel Corn Golden Sweet, Pecan Chips, Pepperoni, Extra Virgin Olive Oil, Hass Avocados, Super Soft Taco Flour Tortillas, Natural Finely Shredded Triple Cheddar Cheese, Grass Fed Angus Ground Beef 93/7, Sunflower Nuts Roasted/No Salt, 100% Pure Vegetable Oil, Mandarins, Greek Vanilla Yogurt, Cinnamon Raisin Bagels)"
3179,"List(Caramel Almond and Sea Salt Nut Bar, White Chocolate Macadamia Nut Energy Bar, Tea Tree Scalp Treatment Shampoo, Sparkling Water Grapefruit, Nuts & Spices, Dark Chocolate Mocha Almond, Honey Bunny Grahams, Lime Sparkling Water, Krinkle Cut Classic Barbecue Potato Chips)"
4092,"List(Organic Baby Spinach, Sharp Cheddar Cheese, Garlic, Mozzarella Cheese, Organic Zucchini, Organic Extra Firm Tofu, Organic Garbanzo Beans, Italian Extra Virgin Olive Oil, Salted Butter, Mint, Kale Greens, Large Lemon, Carrots, Limes, Organic Grape Tomatoes, Parsley, Italian (Flat), New England Grown, Fresh Ginger Root)"


##Train ML Model
To understand the frequency of items are associated with each other (e.g. if anyone order peanut butter, how probability they order jelly), we will use association rule mining for market basket analysis. Spark MLlib implements two algorithms related to frequency pattern mining (FPM): FP-growth and PrefixSpan. The distinction is that FP-growth does not use order information in the itemsets, if any, while PrefixSpan is designed for sequential pattern mining where the itemsets are ordered. We will use FP-growth as the order information is not important for this use case.

In [0]:
from pyspark.ml.fpm import FPGrowth
basket_ds=spark.sql("select items from baskets")
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.001, minConfidence=0)

#creating a model 
model = fpGrowth.fit(basket_ds)
mostPopularItemInABasket=model.freqItemsets
mostPopularItemInABasket.createOrReplaceTempView("mostPopularItemInABasket")


In [0]:
%sql
select items, freq from mostPopularItemInABasket where size(items) > 2 order by freq desc limit 20
--Showing purchased frequency of the products

items,freq
"List(Organic Hass Avocado, Organic Strawberries, Bag of Organic Bananas)",710
"List(Organic Raspberries, Organic Strawberries, Bag of Organic Bananas)",649
"List(Organic Baby Spinach, Organic Strawberries, Bag of Organic Bananas)",587
"List(Organic Raspberries, Organic Hass Avocado, Bag of Organic Bananas)",531
"List(Organic Hass Avocado, Organic Baby Spinach, Bag of Organic Bananas)",497
"List(Organic Avocado, Organic Baby Spinach, Banana)",484
"List(Organic Avocado, Large Lemon, Banana)",477
"List(Limes, Large Lemon, Banana)",452
"List(Organic Cucumber, Organic Strawberries, Bag of Organic Bananas)",424
"List(Limes, Organic Avocado, Large Lemon)",389


## Review Association Rules
In addition to freqItemSets, the FP-growth model also generates association rules. For example, if a shopper purchases peanut butter , what is the likelihood that they will also purchase jelly.
Association rules are created by searching data for frequent if-then patterns and using the criteria support and confidence to identify the most important relationships. Support is an indication of how frequently the itemset appear in the data. Confidence indicates the number of times the if-then statements are found true.

In [0]:
ifThen = model.associationRules
ifThen.createOrReplaceTempView("ifThen")

In [0]:
%sql
select antecedent as `antecedent (if)`,consequent as `consequent (then)`,confidence from ifThen order By confidence desc limit 20

antecedent (if),consequent (then),confidence
"List(Organic Raspberries, Organic Hass Avocado, Organic Strawberries)",List(Bag of Organic Bananas),0.5984251968503937
"List(Organic Cucumber, Organic Hass Avocado, Organic Strawberries)",List(Bag of Organic Bananas),0.546875
"List(Organic Kiwi, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5459770114942529
"List(Organic Navel Orange, Organic Raspberries)",List(Bag of Organic Bananas),0.5412186379928315
"List(Yellow Onions, Strawberries)",List(Banana),0.5357142857142857
"List(Organic Whole String Cheese, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5314685314685315
"List(Organic Navel Orange, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5283018867924528
"List(Organic Raspberries, Organic Hass Avocado)",List(Bag of Organic Bananas),0.521099116781158
"List(Organic D'Anjou Pears, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5170454545454546
"List(Organic Unsweetened Almond Milk, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5141065830721003


### Conclusion


If we analyse the first row of the above table it says **59.84% of customers who bought (Organic Raspberries, Organic Hass Avocado, Organic Strawberries) also bought Bag of Organic Bananas.** From the above result, based on the confidence values provided by the different associtaion rules the items can be arranged in the market to increase sales by  understanding customer purchasing patterns